In [ ]:
#
# Script meant to test the ability to show brief "help" animations in the interactive graph panel
# ... animations still showing incorrect behavior
#
import polars as pl
import rtsvg
rt = rtsvg.RACETrack()

df        = pl.DataFrame({'fm':'a b c d e'.split(), 'to':'b c a c c'.split()})
_relates_ = [('fm','to')]
_pos_     = {'b': (0.57, 0.89), 'a': (0.57, 0.58), 'c': (0.43, 0.72), 'e': (0.43, 0.89), 'd': (0.31, 0.72)}
_rtg_ = rt.interactiveGraphPanel(df, {'relationships':_relates_, 'pos':_pos_}, w=800, h=500)
_rtg_

In [ ]:
#_rtg_.setAnimation('<rect x="10" y="10" width="10" height="10" fill="blue"/>')

In [ ]:
#_rtg_.setAnimation('<text x="5" y="15" fill="black"> popStack </text>')